In [ ]:
!pip install opencv-python pillow numpy matplotlib torch ultralytics ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
import os
import cv2
import PIL
import glob
import torch
import shutil
import numpy as np
from PIL import Image
from ultralytics import YOLO
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
PIL.Image.MAX_IMAGE_PIXELS = None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Main folder where all your data will be stored
main_folder = "/content/drive/MyDrive/Project"  # MODIFY THIS PATH

# Directory for temporary files
temp_folder = os.path.join(main_folder, "temp")

# Output directory for predicted images
output_dir = os.path.join(main_folder, "predictions")

# Path where your trained model will be saved/loaded
model_path = os.path.join(main_folder, "models/best_rooftop.pt")

# Path to your YOLOv8 data.yaml file that defines your dataset
data_yaml_path = os.path.join('/content/drive/MyDrive/rooftop_dataset/data.yaml')
 # MODIFY THIS WITH YOUR DATA.YAML PATH

# Create necessary directories
os.makedirs(temp_folder, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.dirname(model_path), exist_ok=True)

In [ ]:
def cleanup_temp_files():
    """Remove all files in the temporary directory"""
    for file in os.listdir(temp_folder):
        file_path = os.path.join(temp_folder, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")
    print("Temporary files cleaned up")

In [ ]:
def split_image_into_tiles(image, output_folder, tile_size=1024):
    """
    Split a large image into smaller tiles for processing

    Args:
        image: PIL Image object or path to image
        output_folder: Folder to save the tiles
        tile_size: Size of each tile (default: 1024x1024)

    Returns:
        tuple: (number of tiles, number of rows, number of columns)
    """
    # Clean up any existing tiles
    cleanup_temp_files()

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the image if it's a path
    if isinstance(image, str):
        image = Image.open(image)

    width, height = image.size

    # Calculate grid dimensions
    num_cols = (width + tile_size - 1) // tile_size
    num_rows = (height + tile_size - 1) // tile_size

    # Count for naming the tiles
    tile_count = 1

    # Process the image in tiles
    for y in range(0, height, tile_size):
        for x in range(0, width, tile_size):
            # Define boundaries
            right = min(x + tile_size, width)
            bottom = min(y + tile_size, height)

            # Extract tile
            tile = image.crop((x, y, right, bottom))

            # If the tile is smaller than tile_size, pad it
            if tile.size[0] < tile_size or tile.size[1] < tile_size:
                new_tile = Image.new(image.mode, (tile_size, tile_size))
                new_tile.paste(tile, (0, 0))
                tile = new_tile

            # Save the tile
            output_path = os.path.join(output_folder, f"{tile_count}.jpg")
            tile.save(output_path, quality=95)

            tile_count += 1

    return tile_count - 1, num_rows, num_cols

In [ ]:

def train_rooftop_model(data_yaml, epochs=50, imgsz=640):
    """
    Train a YOLOv8 model on rooftop data

    Args:
        data_yaml: Path to the YAML file defining the dataset
        epochs: Number of training epochs
        imgsz: Input image size for training

    Returns:
        Trained model
    """
    print(f"Training model with {data_yaml} for {epochs} epochs...")

    # Initialize model with YOLOv8 segmentation model (pretrained)
    model = YOLO('yolov8s-seg.pt')

    # Train the model (fine-tune on your data)
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        patience=15,  # Early stopping patience
        batch=16,     # Batch size (adjust based on available memory)
        save=True     # Save best model
    )

    # Copy the best model to our specified path
    best_model_path = os.path.join(model.trainer.save_dir, 'weights/best.pt')
    if os.path.exists(best_model_path):
        import shutil
        shutil.copy(best_model_path, model_path)
        print(f"Best model saved to {model_path}")

    return model

In [ ]:
def predict_rooftops_in_tiles(model, tiles_folder, output_folder, conf_threshold=0.25):
    """
    Predict rooftops in all tiles and save B&W masks

    Args:
        model: Loaded YOLO model
        tiles_folder: Folder containing image tiles
        output_folder: Folder to save mask outputs
        conf_threshold: Confidence threshold for predictions
    """
    # Get all JPG images in the tiles folder
    image_files = [
        f for f in os.listdir(tiles_folder)
        if os.path.isfile(os.path.join(tiles_folder, f))
        and f.lower().endswith('.jpg')
    ]

    for image_file in image_files:
        # Load image
        img_path = os.path.join(tiles_folder, image_file)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Could not read image: {img_path}")
            continue

        img_height, img_width = img.shape[:2]

        # Run prediction
        results = model.predict(
            source=img,
            save=False,
            save_txt=False,
            stream=False,
            conf=conf_threshold,
            verbose=False
        )

        # Create a blank mask image
        mask = np.zeros((img_height, img_width), dtype=np.uint8)

        # Process results
        if results[0].masks is not None:
            # Get masks for rooftops (class 0)
            for i, box in enumerate(results[0].boxes):
                if int(box.cls.item()) == 0:  # Check if the class is rooftop (class 0)
                    mask_segment = results[0].masks.data[i].cpu().numpy()
                    mask_segment = cv2.resize(mask_segment.astype(np.uint8), (img_width, img_height))
                    mask = np.logical_or(mask, mask_segment).astype(np.uint8) * 255

        # Save the binary mask
        output_path = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_mask.jpg")
        cv2.imwrite(output_path, mask)

In [ ]:
def merge_masks(tiles_folder, output_path, num_rows, num_cols, original_size=None):
    """
    Merge multiple mask tiles back into a single large image

    Args:
        tiles_folder: Folder containing the mask tiles
        output_path: Path to save the merged mask
        num_rows: Number of rows in the grid
        num_cols: Number of columns in the grid
        original_size: Original image size (width, height) to crop to

    Returns:
        PIL Image of the merged mask
    """
    # Sort images by filename
    mask_files = sorted(
        [f for f in os.listdir(tiles_folder) if f.endswith('_mask.jpg')],
        key=lambda x: int(os.path.splitext(x.split('_')[0])[0])
    )

    # Load the first image to get dimensions
    first_image = Image.open(os.path.join(tiles_folder, mask_files[0]))
    tile_width, tile_height = first_image.size

    # Create a blank canvas for the merged image
    merged_width = num_cols * tile_width
    merged_height = num_rows * tile_height
    merged_image = Image.new('L', (merged_width, merged_height), 0)

    # Place each tile in the correct position
    for idx, file in enumerate(mask_files):
        if idx >= num_rows * num_cols:
            break

        row = idx // num_cols
        col = idx % num_cols

        tile = Image.open(os.path.join(tiles_folder, file))
        merged_image.paste(tile, (col * tile_width, row * tile_height))

    # Crop to original size if specified
    if original_size:
        merged_image = merged_image.crop((0, 0, original_size[0], original_size[1]))

    # Save the merged image
    merged_image.save(output_path)
    return merged_image

In [ ]:
def evaluate_model(model, data_yaml):
    """
    Evaluate model performance on validation data

    Args:
        model: Loaded YOLO model
        data_yaml: Path to the data YAML file

    Returns:
        Metrics from validation
    """
    print(f"Evaluating model performance...")

    # Validate the model
    metrics = model.val(data=data_yaml)

    # Print metrics
    print("\n--- Model Evaluation Metrics ---")
    print(f"Bounding Box mAP50: {metrics.box.map50:.4f}")
    print(f"Bounding Box mAP50-95: {metrics.box.map:.4f}")
    print(f"Segmentation Mask mAP50: {metrics.seg.map50:.4f}")
    print(f"Segmentation Mask mAP50-95: {metrics.seg.map:.4f}")

    return metrics

In [ ]:
def process_uploaded_image(model_path, conf_threshold=0.25):
    """
    Process a user-uploaded image to detect rooftops

    Args:
        model_path: Path to the trained model
        conf_threshold: Confidence threshold for predictions

    Returns:
        Paths to the output files
    """
    # Load the model
    print("Loading model...")
    model = YOLO(model_path)

    # Clean up any existing temporary files
    cleanup_temp_files()

    # Upload image
    print("Please upload a JPG image:")
    uploaded = files.upload()

    if not uploaded:
        print("No image uploaded.")
        return None

    # Get the first uploaded file
    image_filename = list(uploaded.keys())[0]
    image_path = os.path.join(temp_folder, image_filename)

    # Save the uploaded file
    with open(image_path, 'wb') as f:
        f.write(uploaded[image_filename])

    # Load the image
    try:
        image = Image.open(image_path)
        original_width, original_height = image.size
        print(f"Image loaded: {image_filename} ({original_width}x{original_height})")
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

    # For small images, process directly
    if original_width <= 1024 and original_height <= 1024:
        print("Processing small image directly...")

        # Convert to OpenCV format
        img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

        # Run prediction
        results = model.predict(
            source=img_cv,
            save=False,
            save_txt=False,
            stream=False,
            conf=conf_threshold,
            verbose=False
        )

        # Create a blank mask image
        mask = np.zeros((original_height, original_width), dtype=np.uint8)

        # Process results
        if results[0].masks is not None:
            for i, box in enumerate(results[0].boxes):
                if int(box.cls.item()) == 0:  # Check if the class is rooftop (class 0)
                    mask_segment = results[0].masks.data[i].cpu().numpy()
                    mask_segment = cv2.resize(mask_segment.astype(np.uint8), (original_width, original_height))
                    mask = np.logical_or(mask, mask_segment).astype(np.uint8) * 255

        # Save outputs
        mask_path = os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}_mask.jpg")
        cv2.imwrite(mask_path, mask)

        # Create visualization
        colored_mask = np.zeros((original_height, original_width, 3), dtype=np.uint8)
        colored_mask[mask > 0] = [0, 0, 255]  # Red for rooftops

        # Overlay the mask on the original image
        overlay_img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        alpha = 0.5
        overlay = cv2.addWeighted(overlay_img, 1, colored_mask, alpha, 0)

        overlay_path = os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}_overlay.jpg")
        cv2.imwrite(overlay_path, overlay)

    else:
        # For large images, split into tiles
        print("Splitting large image into tiles...")
        num_tiles, num_rows, num_cols = split_image_into_tiles(image, temp_folder)
        print(f"Image split into {num_tiles} tiles ({num_rows}x{num_cols})")

        # Process tiles
        print("Processing tiles...")
        predict_rooftops_in_tiles(model, temp_folder, temp_folder, conf_threshold)

        # Merge tiles
        print("Merging masks...")
        mask_path = os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}_mask.jpg")
        merged_mask = merge_masks(temp_folder, mask_path, num_rows, num_cols, (original_width, original_height))

        # Create visualization
        print("Creating visualization...")
        mask = np.array(merged_mask)
        colored_mask = np.zeros((original_height, original_width, 3), dtype=np.uint8)
        colored_mask[mask > 0] = [0, 0, 255]  # Red for rooftops

        # Overlay the mask on the original image
        overlay_img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        alpha = 0.5
        overlay = cv2.addWeighted(overlay_img, 1, colored_mask, alpha, 0)

        overlay_path = os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}_overlay.jpg")
        cv2.imwrite(overlay_path, overlay)

    # Clean up temporary files
    cleanup_temp_files()

    # Display results
    print("Displaying results...")
    plt.figure(figsize=(20, 10))

    plt.subplot(1, 3, 1)
    plt.title('Original Image')
    plt.axis('off')
    plt.imshow(image)

    plt.subplot(1, 3, 2)
    plt.title('Binary Rooftop Mask')
    plt.axis('off')
    mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    plt.imshow(mask_img, cmap='gray')

    plt.subplot(1, 3, 3)
    plt.title('Overlay Visualization')
    plt.axis('off')
    overlay_img = cv2.imread(overlay_path)
    overlay_img = cv2.cvtColor(overlay_img, cv2.COLOR_BGR2RGB)
    plt.imshow(overlay_img)

    plt.tight_layout()
    plt.show()

    print(f"Results saved to {output_dir}")
    return mask_path, overlay_path

In [ ]:
# Create interactive widgets for model training
def create_training_widgets():
    """Create and display widgets for model training"""
    epochs_slider = widgets.IntSlider(
        value=50, min=10, max=100, step=10,
        description='Epochs:',
        style={'description_width': 'initial'}
    )

    imgsz_slider = widgets.IntSlider(
        value=640, min=320, max=1280, step=64,
        description='Image Size:',
        style={'description_width': 'initial'}
    )

    train_button = widgets.Button(
        description='Train Model',
        button_style='primary',
        tooltip='Click to start training'
    )

    output = widgets.Output()

In [ ]:
train_button = widgets.Button(description="Train Model")

def on_predict_button_clicked(b):
        with output:
            process_uploaded_image(model_path, conf_threshold=conf_slider.value)

predict_button.on_click(on_predict_button_clicked)

    # Display widgets
display(widgets.VBox([widgets.HTML(value="<h3>Rooftop Detection:</h3>"),conf_slider,predict_button,output]))

In [ ]:
model_path

'/content/drive/MyDrive/FinalProject/models/best_rooftop.pt'

In [ ]:
def create_prediction_widgets():
    """Create and display widgets for rooftop prediction"""
    conf_slider = widgets.FloatSlider(
        value=0.25, min=0.05, max=0.95, step=0.05,
        description='Confidence:',
        style={'description_width': 'initial'}
    )

    predict_button = widgets.Button(
        description='Detect Rooftops',
        button_style='success',
        tooltip='Click to upload an image and detect rooftops'
    )

    output = widgets.Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create the button
predict_button = widgets.Button(
    description='Predict',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to run prediction',
    icon='check'  # Optional
)

# Create other UI components (example: confidence slider)
conf_slider = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=1.0,
    step=0.05,
    description='Confidence:',
    continuous_update=False
)

# Output widget to show results
output = widgets.Output()

# Define what happens when button is clicked
def on_predict_button_clicked(b):
    with output:
        output.clear_output()
        process_uploaded_image(model_path, conf_threshold=conf_slider.value)

# Connect the click event
predict_button.on_click(on_predict_button_clicked)

# Display all widgets together
display(widgets.VBox([
    widgets.HTML(value="<h3>Rooftop Detection:</h3>"),
    conf_slider,
    predict_button,
    output
]))


In [ ]:
print("# Rooftop Detection with YOLOv8")
print("This notebook detects rooftops in any JPG image.")
print("You can train the model with your own data or use a pre-trained model.")

# Create training widgets
create_training_widgets()

# Create prediction widgets
create_prediction_widgets()

# Rooftop Detection with YOLOv8
This notebook detects rooftops in any JPG image.
You can train the model with your own data or use a pre-trained model.


In [ ]:
trained_model = train_rooftop_model(data_yaml_path, epochs=50, imgsz=640)
model = YOLO('yolov8s-seg.pt')

Training model with /content/drive/MyDrive/rooftop_dataset/data.yaml for 50 epochs...


100%|██████████| 22.8M/22.8M [00:00<00:00, 222MB/s]


Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/drive/MyDrive/rooftop_dataset/data.yaml, epochs=50, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

100%|██████████| 755k/755k [00:00<00:00, 20.8MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  50%|█████     | 2/4 [00:02<00:02,  1.25s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:06<00:00,  1.58s/it]


                   all        105       2026      0.917      0.887      0.946      0.802       0.92      0.881      0.942      0.752
Speed: 0.2ms preprocess, 7.4ms inference, 0.0ms loss, 5.7ms postprocess per image
Results saved to runs/segment/train
Best model saved to /content/drive/MyDrive/FinalProject/models/best_rooftop.pt


In [ ]:
import torch
print(torch.cuda.is_available())


True
